In [68]:
import numpy as np
import pandas as pd
import zipfile
import pyarrow
import os

Import dataset and get information about the data

In [71]:
train_data = pd.read_parquet(r'C:\Users\nicho\Downloads\Jane street daeseo ai\jane-street-real-time-market-data-forecasting/train.parquet')
train_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 47127338 entries, 0 to 47127337
Data columns (total 93 columns):
 #   Column        Dtype   
---  ------        -----   
 0   date_id       int16   
 1   time_id       int16   
 2   symbol_id     int8    
 3   weight        float32 
 4   feature_00    float32 
 5   feature_01    float32 
 6   feature_02    float32 
 7   feature_03    float32 
 8   feature_04    float32 
 9   feature_05    float32 
 10  feature_06    float32 
 11  feature_07    float32 
 12  feature_08    float32 
 13  feature_09    int8    
 14  feature_10    int8    
 15  feature_11    int16   
 16  feature_12    float32 
 17  feature_13    float32 
 18  feature_14    float32 
 19  feature_15    float32 
 20  feature_16    float32 
 21  feature_17    float32 
 22  feature_18    float32 
 23  feature_19    float32 
 24  feature_20    float32 
 25  feature_21    float32 
 26  feature_22    float32 
 27  feature_23    float32 
 28  feature_24    float32 
 29  feature_25  

In [80]:
def reduce_mem_usage(df, float16_as32=True):
    """ iterate through all the columns of a dataframe and modify the data type
        to reduce memory usage.        
    """
    start_mem = df.memory_usage().sum() / 1024**2
    print('Memory usage of dataframe is {:.2f} MB'.format(start_mem))
    
    for col in df.columns:
        col_type = df[col].dtype
        
        if col_type != object:
            print(df[col])
            c_min = df[col][1:].min()
            c_max = df[col][1:].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col][1:] = df[col][1:].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col][1:] = df[col][1:].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col][1:] = df[col][1:].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col][1:] = df[col][1:].astype(np.int64)  
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col][1:] = df[col][1:].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col][1:] = df[col][1:].astype(np.float32)
                else:
                    df[col][1:] = df[col][1:].astype(np.float64)
        else:
            if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16):
                if float16_as32:
                    df[col][1:] = df[col][1:].astype(np.float32)
                else:
                    df[col][1:] = df[col][1:].astype(np.float16)
            elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                df[col][1:] = df[col][1:].astype(np.float32)
            else:
                df[col][1:] = df[col][1:].astype(np.float64)


    end_mem = df.memory_usage().sum() / 1024**2
    print('Memory usage after optimization is: {:.2f} MB'.format(end_mem))
    print('Decreased by {:.1f}%'.format(100 * (start_mem - end_mem) / start_mem))
    
    return df


# def import_data(file):
#     """create a dataframe and optimize its memory usage"""
#     df = pd.read_csv(file, parse_dates=True, keep_date_col=True)
#     df = reduce_mem_usage(df)
#     return df

In [81]:
train_data = reduce_mem_usage(train_data, False)

Memory usage of dataframe is 8179.83 MB
0              0
1              0
2              0
3              0
4              0
            ... 
47127333    1698
47127334    1698
47127335    1698
47127336    1698
47127337    1698
Name: date_id, Length: 47127338, dtype: int16
0             0
1             0
2             0
3             0
4             0
           ... 
47127333    967
47127334    967
47127335    967
47127336    967
47127337    967
Name: time_id, Length: 47127338, dtype: int16
0            1
1            7
2            9
3           10
4           14
            ..
47127333    34
47127334    35
47127335    36
47127336    37
47127337    38
Name: symbol_id, Length: 47127338, dtype: int8
0           3.888672
1           1.371094
2           2.285156
3           0.690430
4           0.440674
              ...   
47127333    3.242188
47127334    1.079102
47127335    1.033203
47127336    1.243164
47127337    3.193359
Name: weight, Length: 47127338, dtype: float16


C:\Users\nicho\AppData\Local\Temp\ipykernel_11720\4241687177.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[col][1:] = df[col][1:].astype(np.int16)
C:\Users\nicho\AppData\Local\Temp\ipykernel_11720\4241687177.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[col][1:] = df[col][1:].astype(np.int16)
C:\Users\nicho\AppData\Local\Temp\ipykernel_11720\4241687177.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[col][1:]

0                NaN
1                NaN
2                NaN
3                NaN
4                NaN
              ...   
47127333    2.525391
47127334    1.857422
47127335    2.515625
47127336    2.664062
47127337    2.728516
Name: feature_00, Length: 47127338, dtype: float16


C:\Users\nicho\AppData\Local\Temp\ipykernel_11720\4241687177.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[col][1:] = df[col][1:].astype(np.float16)
c:\Users\nicho\AppData\Local\Programs\Python\Python312\Lib\site-packages\pandas\io\formats\format.py:1458: RuntimeWarning: overflow encountered in cast
  has_large_values = (abs_vals > 1e6).any()


0                NaN
1                NaN
2                NaN
3                NaN
4                NaN
              ...   
47127333   -0.722168
47127334   -0.790527
47127335   -0.672363
47127336   -0.889160
47127337   -0.745117
Name: feature_01, Length: 47127338, dtype: float16


...

0           0.346924
1           0.216675
2           0.670898
3           0.775391
4          -1.088867
              ...   
47127333   -0.022430
47127334   -0.123596
47127335   -0.000367
47127336   -0.026581
47127337   -0.038757
Name: responder_7, Length: 47127338, dtype: float16


C:\Users\nicho\AppData\Local\Temp\ipykernel_11720\4241687177.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[col][1:] = df[col][1:].astype(np.float16)
c:\Users\nicho\AppData\Local\Programs\Python\Python312\Lib\site-packages\pandas\io\formats\format.py:1458: RuntimeWarning: overflow encountered in cast
  has_large_values = (abs_vals > 1e6).any()


0           0.095520
1           0.778809
2           0.772949
3          -1.379883
4          -5.000000
              ...   
47127333   -0.252441
47127334   -0.460693
47127335   -0.125122
47127336   -0.256348
47127337   -0.211914
Name: responder_8, Length: 47127338, dtype: float16
0           0
1           0
2           0
3           0
4           0
           ..
47127333    9
47127334    9
47127335    9
47127336    9
47127337    9
Name: partition_id, Length: 47127338, dtype: category
Categories (10, int32): [0, 1, 2, 3, ..., 6, 7, 8, 9]


C:\Users\nicho\AppData\Local\Temp\ipykernel_11720\4241687177.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[col][1:] = df[col][1:].astype(np.float16)


TypeError: Categorical is not ordered for operation min
you can use .as_ordered() to change the Categorical to an ordered one


In [ ]:
print(train_data['date_id'].unique())
print(np.max(train_data['feature_00'].unique()))
print(np.min(train_data['feature_00'].unique()))

[   0    1    2 ... 1696 1697 1698]
6.477002


In [ ]:
feat_cols = [f'feature_{i:02d}' for i in range(79)]
print(feat_cols)
for i in feat_cols:
    train_data[i][np.isnan(train_data[i])] = 0
print([np.max(train_data[i]) for i in feat_cols])
print([np.min(train_data[i]) for i in feat_cols])

['feature_00', 'feature_01', 'feature_02', 'feature_03', 'feature_04', 'feature_05', 'feature_06', 'feature_07', 'feature_08', 'feature_09', 'feature_10', 'feature_11', 'feature_12', 'feature_13', 'feature_14', 'feature_15', 'feature_16', 'feature_17', 'feature_18', 'feature_19', 'feature_20', 'feature_21', 'feature_22', 'feature_23', 'feature_24', 'feature_25', 'feature_26', 'feature_27', 'feature_28', 'feature_29', 'feature_30', 'feature_31', 'feature_32', 'feature_33', 'feature_34', 'feature_35', 'feature_36', 'feature_37', 'feature_38', 'feature_39', 'feature_40', 'feature_41', 'feature_42', 'feature_43', 'feature_44', 'feature_45', 'feature_46', 'feature_47', 'feature_48', 'feature_49', 'feature_50', 'feature_51', 'feature_52', 'feature_53', 'feature_54', 'feature_55', 'feature_56', 'feature_57', 'feature_58', 'feature_59', 'feature_60', 'feature_61', 'feature_62', 'feature_63', 'feature_64', 'feature_65', 'feature_66', 'feature_67', 'feature_68', 'feature_69', 'feature_70', 'feat

C:\Users\nicho\AppData\Local\Temp\ipykernel_11720\4116330321.py:4: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure this keeps updating the original `df`.

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

  train_data[i][np.isnan(train_data[i])] = 0
C:\Users\nicho\AppData\Local\Temp\ipykernel_11720\4116330321.py:4: SettingWithCopyWar

[np.float32(6.477002), np.float32(6.2923913), np.float32(6.490265), np.float32(6.695623), np.float32(6.16416), np.float32(35.726223), np.float32(93.64406), np.float32(54.94161), np.float32(21.695173), np.int8(82), np.int8(12), np.int16(539), np.float32(139.41841), np.float32(381.20224), np.float32(169.11882), np.float32(111.07955), np.float32(220.13248), np.float32(134.50064), np.float32(5.8253703), np.float32(5.9723926), np.float32(5.421159), np.float32(161.48677), np.float32(4.128449), np.float32(3.24372), np.float32(10.7444515), np.float32(4.136297), np.float32(4.862733), np.float32(2.6435504), np.float32(2.88091), np.float32(7.5347056), np.float32(9.410683), np.float32(127.98037), np.float32(6.007876), np.float32(5.996757), np.float32(5.0699673), np.float32(5.371406), np.float32(5.7007275), np.float32(31.50553), np.float32(26.22109), np.float32(5.585663), np.float32(5.4734793), np.float32(5.2480097), np.float32(5.1484833), np.float32(5.151497), np.float32(4.9408584), np.float32(5.2

In [64]:
# Log Transform and normalize it to 0,1
for i in feat_cols:
    mean_val = np.mean(train_data[i])
    std_val = np.std(train_data[i])
    train_data[i] = (train_data[i] - mean_val) / std_val

# Normalize log-transformed data to [0, 1] range

print(train_data)

          date_id  time_id  symbol_id    weight  feature_00  feature_01  \
0               0        0          1  3.889038         NaN         NaN   
1               0        0          7  1.370613         NaN         NaN   
2               0        0          9  2.285698         NaN         NaN   
3               0        0         10  0.690606         NaN         NaN   
4               0        0         14  0.440570         NaN         NaN   
...           ...      ...        ...       ...         ...         ...   
47127333     1698      967         34  3.242493         NaN         NaN   
47127334     1698      967         35  1.079139         NaN         NaN   
47127335     1698      967         36  1.033172         NaN         NaN   
47127336     1698      967         37  1.243116         NaN         NaN   
47127337     1698      967         38  3.193685         NaN         NaN   

          feature_02  feature_03  feature_04  feature_05  ...  responder_0  \
0                NaN 